In [ ]:
from model import get_device, get_restnet152, make_bow_vector, make_target, PrincipalClassificator, ImageClassificator, BoWClassifier, CNN, LSTMTagger,  ModelMix
from torch import nn, optim
from data_loader import load_split_data
from torch.optim import SGD

import json
import torch

In [ ]:
BATCH_SIZE = 32
LEARN_RATE = 0.05
EMBEDDING_DIM = 200

with open('./datainfo.json', 'r') as f:
    data = json.load(f)  
    
vocab = data["vocab"]
vocab_size = len(vocab)

trainloader, testloader = load_split_data(data, BATCH_SIZE, .2)

In [ ]:
device = get_device()
model1 = LSTMTagger(EMBEDDING_DIM, vocab_size, 3)
model2 = CNN()
model = ModelMix(model1, model2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=LEARN_RATE)

In [ ]:
trainloader, testloader = load_split_data(data, BATCH_SIZE, .2)

In [ ]:
import PIL
import os  


def train_bow():

    epochs = 5
    steps = 0
    running_loss = 0
    print_every = 10
    train_losses, test_losses = [], []
    
    classes = {0: 0, 1: 1, 2: 2}
    

    for epoch in range(epochs):
        try :
            for image, text, labels in trainloader:
                steps += 1                
            
                bow_vec = torch.stack(make_bow_vector(text, vocab)).cuda()
                target = torch.stack(make_target(labels.tolist(), classes)).cuda()
                labels = labels.to(device)
                
                optimizer.zero_grad()
                predict = model_text.forward(bow_vec)

                loss = criterion(predict.squeeze(1), target.squeeze(1))
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                
                if steps % print_every == 0:
                    test_loss = 0
                    accuracy = 0
                    total = 0
                    model.eval()
                    with torch.no_grad():
                        for image, text, labels in testloader:
                            
                            bow_vec = torch.stack(make_bow_vector(text, vocab)).to(device)
                            target = torch.stack(make_target(labels.tolist(), classes)).to(device)
                            labels = labels.to(device)
                            
                            predict = model_text.forward(bow_vec)
                            batch_loss = criterion(predict.squeeze(1), target.squeeze(1))
                            test_loss += batch_loss.item()
                                                        
                            val, ind = predict.squeeze(1).max(1) 
                            accuracy += (ind == labels).sum()
                            total += len(labels)

                    train_losses.append(running_loss/len(trainloader))
                    test_losses.append(test_loss/len(testloader))                    
                    print(f"Epoch {epoch+1}/{epochs}.. "
                        f"Train loss: {running_loss/print_every:.3f}.. "
                        f"Test loss: {test_loss/len(testloader):.3f}.. "
                        f"Test accuracy: {float(accuracy)/float(total):.3f}")
                    running_loss = 0
                    accuracy = 0
                    total =0
                    model.train()
        except PIL.UnidentifiedImageError as error:
            print(error)
            er = str(error).split("'")
            os.remove(er[1])
            uwu()
        
#train_bow()

In [ ]:
from model import get_device, get_restnet152, make_bow_vector, make_target, PrincipalClassificator, ImageClassificator, BoWClassifier, CNN, LSTMTagger,  ModelMix
from torch import nn, optim
from data_loader import load_split_data
from torch.optim import SGD
from torch.utils.tensorboard import SummaryWriter
from trainModel import Train

import json
import torch



In [ ]:
BATCH_SIZE = 32
LEARN_RATE = 0.05
EMBEDDING_DIM = 200

with open('./datainfo.json', 'r') as f:
    data = json.load(f)  
    
vocab = data["vocab"]
vocab_size = len(vocab)
writer = SummaryWriter(comment=f"LSTM {LEARN_RATE} {BATCH_SIZE}")

trainloader, testloader = load_split_data(data, BATCH_SIZE, .2)

In [ ]:
device = get_device()
model1 = LSTMTagger(EMBEDDING_DIM, vocab_size, 3)
model2 = CNN()
model = ModelMix(model1, model2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=LEARN_RATE)

experiment = Train(model, optimizer, criterion,trainloader, testloader, writer = writer)
experiment.train_model()
